In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from utils.useful_tool import convert_polar_df, fetch_freezing_frames

In [2]:
intervals = ['1h', '6h']
polar_dfs = None

for interval in intervals:
    for animal_number in range(1, 5):
        filepath = './resources/extinction/dlc/60min_x1_%s_EXT_%d.csv' % (interval, animal_number)
        df = pd.read_csv(filepath, header=[1, 2], index_col=[0])

        bodyparts = list(set([body for (body, _) in df.columns]))
        drop_columns = [(body_name, 'likelihood') for body_name in bodyparts]
        df.drop(drop_columns, axis=1, inplace=True)

        xy_df = pd.read_csv('./resources/extinction/dlc/60min_x1_%s_EXT_%d_XY.tsv' % (interval, animal_number), sep="\t", header=[34], index_col=[0])
        freezing_series = fetch_freezing_frames(xy_df)

        polar_df = convert_polar_df(df, xy_df)
        polar_df = polar_df.apply(lambda x: (x - x.mean()) / x.std())
        polar_df[(None, 'FZ')] = freezing_series
        
        if polar_dfs is None:
            polar_dfs = polar_df.copy(deep=True)
        else:
            polar_dfs = pd.concat([polar_dfs, polar_df.copy(deep=True)])

In [4]:
polar_dfs

bodyparts      nose             leftear            rightear            \
coords     distance     theta  distance     theta  distance     theta   
0          0.714395  1.614163  0.715800  1.124393 -0.401242  1.342285   
1          0.696176 -1.864340  0.706776  1.149453 -0.561859  1.397964   
2          0.629551 -1.837609  0.591426  1.166265 -0.649141  1.420906   
3          0.571614 -1.841904  0.533986  1.140497 -0.744720  1.391328   
4          0.587040 -1.847379  0.789662  1.225774 -0.686432  1.404708   
...             ...       ...       ...       ...       ...       ...   
7196       0.062458  0.125287 -0.925582 -0.264227 -1.306559  0.206460   
7197       0.181903  0.127608 -0.868423 -0.262802 -1.406007  0.227603   
7198       0.044782  0.126870 -0.677283 -0.228485 -0.976874  0.207629   
7199       0.740283 -0.011220 -0.408400 -0.308697 -0.658996  0.086981   
7200       0.028209  0.082333 -0.790674 -0.280932 -1.316828  0.152763   

bodyparts      neck               body1            ...     tail1            \
coords     distance     theta  distance     theta  ...  distance     theta   
0          0.033256  1.102174 -0.224015  0.795599  ...  0.204624 -0.213589   
1         -0.078462  1.135936 -0.310176  0.865914  ...  0.173442 -0.210717   
2         -0.127044  1.164392 -0.345410  0.902963  ...  0.199736 -0.213896   
3         -0.134747  1.127074 -0.318903  0.841999  ...  0.028930 -0.204101   
4         -0.089280  1.149278 -0.269479  0.868159  ...  0.018204 -0.201215   
...             ...       ...       ...       ...  ...       ...       ...   
7196      -1.174330 -0.041897 -1.469120 -0.171356  ...  1.887175 -1.353919   
7197      -1.119853 -0.036840 -1.356287 -0.180541  ...  1.531413 -1.348541   
7198      -0.911021 -0.008760 -1.151109 -0.115235  ...  1.457853 -1.341679   
7199      -0.852618 -0.122458 -1.336071 -0.106213  ...  2.160339 -1.384834   
7200      -1.282659 -0.122829 -1.529067 -0.212991  ...  1.944620 -1.362808   

bodyparts     tail2               tail3               tail4            \
coords     distance     theta  distance     theta  distance     theta   
0          0.087561 -0.350636  0.156935 -0.583385 -0.168325 -0.851239   
1          0.055407 -0.347504  0.184699 -0.582419 -0.169963 -0.847897   
2          0.041358 -0.349700  0.189431 -0.582851 -0.177869 -0.844174   
3         -0.053563 -0.345525  0.138281 -0.582508 -0.204445 -0.844194   
4         -0.075707 -0.344337  0.134820 -0.580871 -0.207640 -0.843750   
...             ...       ...       ...       ...       ...       ...   
7196       1.178160 -1.417861  1.048240 -1.507556  0.434871  1.852671   
7197       1.032686 -1.414749  0.953392 -1.505996  0.376324  1.851711   
7198       0.964232 -1.407471  0.770090 -1.499649  0.318976  1.865487   
7199       1.265479 -1.433663  1.106342  1.900762  0.507513  1.852609   
7200       1.177387 -1.421142  1.047379 -1.507077  0.454329  1.858538   

bodyparts               NaN     
coords    distance_by_frame FZ  
0                 -0.528614  0  
1                 -0.449737  1  
2                 -0.449737  1  
3                 -0.528614  1  
4                 -0.528614  1  
...                     ... ..  
7196              -0.635958  0  
7197              -0.635958  0  
7198              -0.526955  0  
7199              -0.199947  0  
7200              -0.526955  0  

[57608 rows x 26 columns]

In [5]:
polar_dfs = polar_dfs.apply(lambda x: (x-x.mean()) / x.std(), axis=0)
data = polar_dfs.to_numpy()
pca = PCA()
pca.fit(data)
feature = pca.transform(data)

print(pca.explained_variance_ratio_)

pca_list = feature[:, :6]
tsne = TSNE(n_components=2, metric='cosine')
x_embedded = tsne.fit_transform(pca_list)
x_embedded

x, y = zip(*x_embedded)

plt.figure(figsize=(8, 8), dpi=200)
plt.scatter(x, y, alpha=0.5, s=1.0)
plt.grid()
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()

[0.26726764 0.17994985 0.09924028 0.07023186 0.05249155 0.04735381
 0.04203444 0.03372953 0.02697879 0.02493174 0.02335888 0.02115086
 0.01751268 0.01523103 0.01337787 0.00934033 0.00812846 0.00796241
 0.00767251 0.00702199 0.00613857 0.00547543 0.00435411 0.0036204
 0.00352923 0.00191576]


/opt/conda/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(
